# RawDataReader Example

In [ ]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import re
import pandas as pd 
import numpy as np
import copy
import os

from SWAI.IO.BinFileReader.config import *
from SWAI.IO.BinFileReader.MainPackageReader import MainPackageReader
from SWAI.IO.BinFileReader.BaseSensorData import SensorDataUltrasonic, SensorDataTime, SensorDataBase
from SWAI.IO.BinFileReader.DataPackage import *
from SWAI.IO.SensorData.SensorDataBase import SensorDataBase
from SWAI.IO.SensorData.SensorUltrasonic import SensorDataUltrasonic
from SWAI.IO.SensorData.SensorDataTime import SensorDataTime

In [ ]:
def check_samplingrates(d_amount_pkgs):
    # check the sampling rates for the different Sensors
    # and prints them out
    t_s = d_amount_pkgs['timestamp'] /10
    print("Samplingrates")
    print("-----------------")
    for key in d_amount_pkgs.keys():
        num_values = d_amount_pkgs[key]
        if key in ['US_front','US_back']:
            num_values = num_values * 64
        sample_rate = num_values/t_s
        print("{:<20} {:>10}".format(key +':', round(sample_rate,1)))

## Read out the raw file

In [ ]:
#### config
# create list of all packages which are expected
l_packages = [
              pkg_timestamp,  
              pkg_adxl_front,
              pkg_adxl_back,
              pkg_adis_front,
              pkg_adis_back,
              pkg_iis3_front,
              pkg_iis3_back,
              pkg_us_front,
              pkg_us_back,
              pkg_rpm,
              pkg_speed_backwheel,
              pkg_oiltemp,
              pkg_motor_temp,
              pkg_gear
              ]

In [ ]:
path_bin_file = r"U:\01_Datenbanken\12_example_files\example_bin_file_reader\data_5.bin"
with open(path_bin_file, "rb") as file:
    raw_file = file.read()  

In [ ]:
# create a Package Reader object for reading out the raw file
raw_file_reader = MainPackageReader(l_subpackages=l_packages)

input_file = raw_file # [0:100000000]
data = raw_file_reader.read_out_raw_file(input_file)

print("amount packages")
raw_file_reader.d_amount_pkgs

In [ ]:
check_samplingrates(raw_file_reader.d_amount_pkgs)

In [ ]:
df = raw_file_reader.convert_data_to_pandas(l_data=data)

## Time

In [ ]:
sensor_data_time = SensorDataTime(sensorname='Timestamp')
np_abs_time, l_steps = sensor_data_time.calc_abs_timestamps(l_timestamps = list(df['timestamp']['timestamp']))
%matplotlib inline
fig, ax = plt.subplots(1,2,figsize=(10,3))
ax[0].plot(np_abs_time)
ax[0].set_title("increasing of the abs time")
ax[1].plot(l_steps)
ax[1].set_title("steps between two packages")

## Vibration

In [ ]:
# Plot all the vibration data
%matplotlib qt

l_sensorname = ['IIS3_front','IIS3_back','ADXL_front','ADXL_back', 'ADIS_front', 'ADIS_back']
for sensorname in l_sensorname:
    data_x = df[sensorname]['X']
    data_y = df[sensorname]['Y']
    data_z = df[sensorname]['Z']
    sensor_data = SensorDataBase(sensorname=sensorname)
    data_x = sensor_data.remove_none_values(data_x)
    data_y = sensor_data.remove_none_values(data_y)
    data_z = sensor_data.remove_none_values(data_z)
    plt.figure()
    plt.plot(data_x)
    plt.plot(data_y)
    plt.plot(data_z)
    plt.title(sensorname)
    plt.legend(['x','y','z'])

## Ultrasonic

In [ ]:
# create ultrasonic data object
sensor_data_us = SensorDataUltrasonic(sensorname='ultrasonic')

sensor_data_us = SensorDataUltrasonic(sensorname='ultrasonic')
l_data_keys = ['US_front', 'US_back']
l_data =[]
for key in l_data_keys:
    l_us = list(df[key]['US'])
    l_us = sensor_data_us.correct_msb_error(l_us)
    l_us = sensor_data_us.flatten_of_data(l_us)
    np_us = sensor_data_us.remove_none_values(l_us)
    l_data.append(np_us)

if 1:
    SensorDataUltrasonic.convert_to_wav_file(np_us, 
                                             file_path= path_bin_file.split(".")[0] + '.wav')

## Visualization

In [ ]:
%matplotlib qt

start_idx = 192000 * 250
end_idx = 192000 * 400

plt.figure()
plt.plot(l_data[0][start_idx:end_idx])
plt.plot(l_data[1][start_idx:end_idx])
plt.legend([l_data_keys[0], l_data_keys[1]])